In [326]:
from mesa import Agent, Model
from mesa.time import RandomActivation, BaseScheduler
from mesa.datacollection import DataCollector

from scipy.stats import powerlaw, lognorm
import numpy as np
import pandas as pd
import powerlaw as pr
from numpy.random import normal
import random
from operator import attrgetter
from ddpg import Agent, ReplayBuffer, OUActionNoise, CriticNetwork, ActorNetwork

import torch

#Initialize the reinforcement learning model 
agent = Agent(alpha = 0.001, beta = 0.001, input_dims = [12], tau = 0.001, env = 1, batch_size = 64, layer1_size = 400, layer2_size = 300, n_actions=1)

##VC Coefficients
#VC Coefficients - General
Number_of_VCs = 100 #approximate number of VCs in USA. In fact, it is 1000, but for computaional reasons we devide everything by 10
Fund_maturity = 40 #the number of time steps until the outcome of the fund is realised, 1 step=3months
VC_quality_alpha = 2.06 #alpha coefficient for power law distribution
VC_quality_x_min = 0 #x_min coefficient for power law distribution
VC_max_TVPI = 14 #we want to normalize VC_quality so that it lies between 0 and 1
Average_portfolio_size = 32 #Based on real world data

#VC Coefficients - Employees
Number_of_employees_sd = 1.3711 #standard deviation coefficinet for lognormal distribution of number of employees
Number_of_employees_loc = 0.8426 #loc coefficient for lognormal distribution of number of employees
Number_of_employees_scale = 9.5626 #scale coefficient for lognormal distribution of number of employees
VC_work_hours_per_week = 56 #Average numebr of hours worked by an analyst in VC
Work_weeks_per_month = 4 
Work_hours_per_month = VC_work_hours_per_week*Work_weeks_per_month #Work hours per months per employee in VC
Work_hours_per_3months = Work_hours_per_month*3 #1 time step = 3 months, thus we are interested in hours per 3 months
Percentage_of_time_spend_on_other_activities = 0.31 #time spend by VC employee on activitties not related to either screening or advising
Time_for_screening_and_monitroring_3months_per_emp = Work_hours_per_3months*(1-Percentage_of_time_spend_on_other_activities)
Number_of_funds_at_operation = 2 #at same time, VC takes care of multiple funds
Time_for_screening_and_monitroring_3months_per_emp_per_fund = Time_for_screening_and_monitroring_3months_per_emp/Number_of_funds_at_operation
Average_number_of_investemnt_analysts = 19 #Based on real-world data 

#VC Coefficients - Time needed
Screening_time = 60 #Time in hours needed to screen a startup
Advising_time = 27.5 #Time in hours needed per time step(i.e. 3 months) to advise to a startup in the portfolio 

#VC Coefficients - Returns
Early_returns_alpha = 2.3758 #alpha coefficient for power law distribution of early stage retruns
Early_returns_x_min = 4.5761 #X_min coefficeint for power law distribution of early stage returns
Early_startup_exit = 32 #number of time steps it takes early startup to exit
Late_returns_lognormal_sd = 0.98981 #standard deviation for log normal distribution of late stage returns
Late_returns_lognormal_loc = -0.133236 #location coefficient for log normal distribution of late stage returns
Late_returns_lognormal_scale = 1.79759 #scale coefficinet for log normal distribution of late stage returns
Late_startup_exit = 24 #number of time steps it takes a late stage startup to exit

##Startup Coefficients
#Startup Coefficients - General
Number_of_new_startups = 25600 #Number of business starts in USA every 3 months, In fact, it is 256000, but for computaional reasons we devide everything by 10
Number_of_late_stage_startups = 36 #It is in fact 355, but we divide everything by 10 for computational reasons
EPI_alpha = 0.29872 #alpha coefficient for power law distribution of EPI which is taken as a proxy for startup potential
EPI_loc = 0 #location coefficient for power law distribution of EPI
EPI_scale = 1 #scale coefficient for power law distribtuion of EPI
Percentage_of_startups_getting_to_later_stage = 0.2979 #Percentage of startups which recieve funding in 1st and 2nd round 

#Startup Coefficients - Time progression equation
#EPI = Alpha*EPI + Beat*VC + Industry_Shock + Macro_Shock + Idiosyncratic_shock
Alpha = 0.99 #alpha coefficient for time progression equation. Expresses weight of EPI
Beta = 0.01 #beta coefficient for time progression equation. Expresses the weight of VC 
Macro_shock_mean = 0 #mean for normal distribution of macro shock
Macro_shock_sd = 0.0681 #standard deviation for normal distribution of macro shock
Industry_shock_mean = 0 #mean for normal distribution of industry shock
Industry_shock_sd = 0.0432 #standard deviatio for normal distribution of industry shock
Idiosyncratic_shock_mean = 0 #mean for normal distribution for idiosyncratic shock
Idiosyncratic_shock_sd = 0.0775 #standard deviation for normal distribtion for idiosyncratic shock

#Startup Coefficeints - Industries
List_of_Industries = ["Business Productivity Software", "Drug Discovery", "Financial Software", "Media and Infromation Services", "Network Management Software", "Biotechnology", "Application Software", "Therapeutic Devices", "Surgical Devices", "Other Healthcare Tech Systems"]
Probability_Distribution_of_Industries = [0.3486, 0.1133, 0.10841, 0.1006, 0.0844, 0.0729, 0.0556, 0.0440, 0.0365, 0.0357]
Correlation_matrix = pd.read_excel("Correlation_matrix.xlsx")
Correlation_matrix.index = Correlation_matrix["Unnamed: 0"]
del Correlation_matrix["Unnamed: 0"]

#Startup Coefficients - Screening
Noise_mean_before_screening_ES = 0 #mean for normal distribution of noise before screening for early stage startups
Noise_standard_deviation_before_screening_ES = 0.75 #standard deviation for normal distribution of noise before screening for early stage startups
Noise_mean_after_screening_ES = 0 #mean for normal distribution of noise after screening for early stage startups
Noise_standard_deviation_after_screening_ES = 0.2 #standard deviation for normal distribution of noise after screening for early stage startups
Noise_mean_before_screnniing_LS = 0 #mean for normal distribution of noise before screening for late stage startups
Noise_standard_deviation_before_screening_LS = 0.53 #standard deviation for normal distribution of noise before screening for late stage startups
Noise_mean_after_screening_LS = 0 #mean for normal distribution of noise after screening for late stage startups
Noise_mean_before_screnniing_LS = 0.14 #standard deviation for normal distribution of noise after screening for late stage startups

#Startup Coefficients - Investors
Number_of_investors_per_round = 5 #max number of investors allowed to invest in startup
Number_of_due_diligence_investors = 10 #number of investors enagaged in due diligence

##A sample of early stage returns, used for potential -> return mapping
#powerlaw package does not have an inverse cdf function, hence we apporximate it with a sample
sample_size = 10000
theoretical_distribution_ER = pr.Power_Law(x_min = Early_returns_x_min, parameters = [Early_returns_alpha])
simulated_data_ER = theoretical_distribution_ER.generate_random(sample_size)
simulated_data_new_ER = []
for i in simulated_data_ER: #simulated data start with 1, but rerturns start with 0, hence we translate the smaple by -1
    i = i-1
    simulated_data_new_ER.append(i)
sample_data_ER = sorted(simulated_data_new_ER)

##A distribution for VC_quality
theoretical_distribution_VC = pr.Power_Law(x_min = VC_quality_x_min, parameters = [VC_quality_alpha])

##General model coefficents
Risk_free_rate = 1.521 #10 year return on us treasury bill
Estimate_of_early_stage_screenings = int((Number_of_VCs * Average_number_of_investemnt_analysts * (Time_for_screening_and_monitroring_3months_per_emp_per_fund/Screening_time)*(1-Percentage_of_startups_getting_to_later_stage))/(Number_of_due_diligence_investors))

In [327]:
##Here we define class for VC, Startup and Activation
#VC is assigend a unique id, VC quality and the number of investment analysts
class VC(Agent):
    def __init__(self, unique_id, VC_quality, Investment_analysts, Fund_life_stage, model):
        self.unique_id = unique_id
        self.model = model
        self.VC_quality = VC_quality
        self.Fund_life_stage = Fund_life_stage
        self.Investment_analysts = Investment_analysts
        self.Endowement = 1
        self.Screening_prospects = []
        self.Portfolio = []
        self.Portfolio_size = len(self.Portfolio)
        self.Effort_left_for_fund = self.Investment_analysts*Time_for_screening_and_monitroring_3months_per_emp_per_fund
        self.Effort_allocated_to_startups_in_portfolio = self.Portfolio_size*Advising_time
        self.Effort_left_for_screening = self.Effort_left_for_fund - self.Effort_allocated_to_startups_in_portfolio
        self.Number_of_available_screenings = self.Effort_left_for_screening/Screening_time 
        self.Time_till_end_of_investment_stage_E = max(0, (Fund_maturity - Early_startup_exit - self.Fund_life_stage))
        self.Time_till_end_of_investment_stage_L = max(0, (Fund_maturity - Late_startup_exit - self.Fund_life_stage))
        
    #This funciton enables us to map EPI (startup potnetial) into returns
    def EPI_to_returns(self, EPI, stage):
        #This gives us probability of observing EPI less or equal to observed vlaue
        probability = powerlaw.cdf(EPI, EPI_alpha, EPI_loc, EPI_scale)
        #If VC has invested in the startup in early stage then:
        if stage == "Early":
            return float(sample_data_ER[int(sample_size*probability)])
        #If VC has invested in the startup in later stage then:
        if stage == "Late":
            return float(lognorm.ppf(probability, Late_returns_lognormal_sd, Late_returns_lognormal_loc, Late_returns_lognormal_scale))        
    
    #Projects EPI fo startups into the future
    def time_progression_projected(self, EPI):
        EPI_ = Alpha*EPI + Beta*self.VC_quality + np.random.normal(Idiosyncratic_shock_mean, Idiosyncratic_shock_sd)\
        + np.random.normal(Industry_shock_mean, Industry_shock_sd) + np.random.normal(Macro_shock_mean, Macro_shock_sd)
        return EPI_

    #Calculates expecetd covariance between two startups
    #Since EPI_final boils down to EPI_final ≈ Alpha^32*EPI_0 + (Alpha^31 + Alpha^30 + ...+ 1) *Beta*Advising
    #+(Alpha^31 + Alpha^30 + ...+ 1) * Average_Idiosyncratic_shock + (Alpha^31 + Alpha^30 + ...+ 1) *Averga_Industry_shock +(Alpha^31 + Alpha^30 + ...+ 1) *Averga_Macro_shock
    # the Cov(Startup_1, Startup_2) = 
    def expected_covariance(self, startup_1, startup_2):
        time_together = Early_startup_exit - max(getattr(startup_1, "Life_stage"), getattr(startup_2, "Life_stage"))
        alpha_sumation_1 = Alpha**(1 + Early_startup_exit - getattr(startup_1, "Life_stage")+time_together)*((1 - Alpha^(1+Early_startup_exit-getattr(startup_1, "Life_stage")))/(1-Alpha))
        alpha_sumation_2 = Alpha**(1 + Early_startup_exit - getattr(startup_2, "Life_stage")+time_together)*((1 - Alpha^(1+Early_startup_exit-getattr(startup_2, "Life_stage")))/(1-Alpha))   
        Industry_correlation =  Correlation_matrix.loc[getattr(startup_1, "Industry"), getattr(startup_1, "Industry")] 
        return alpha_sumation_1*alpha_sumation_2*Industry_correlation*Industry_shock_sd^2 + alpha_sumation_1*alpha_sumation_2
    
    #Calculate variance for one startup
    def expected_variance(self, startup_1):
        alpha_sumation_1 = ((1 - Alpha**(Early_startup_exit - getattr(startup_1, "Life_stage")))/(1-Alpha))
        alpha_sumation_2 = ((1 - Alpha**(1 + Early_startup_exit - getattr(startup_1, "Life_stage")))/(1-Alpha))                   
        if getattr(startup_1, "Life_stage") == 0:
            noise = Noise_standard_deviation_after_screening_ES
        else:
            noise = Noise_standard_deviation_after_screening_ES/(getattr(startup_1, "Life_stage")**(1/2))
        return alpha_sumation_1**2*noise + alpha_sumation_2**2*Idiosyncratic_shock_sd + alpha_sumation_2**2*Industry_shock_sd + alpha_sumation_2**2*Macro_shock_sd                                                                                         

    #Calculate expected variance for the whole portfolio 
    def expected_portfolio_variance(self, Portfolio):
        if len(Portfolio) <=1:
            return 0
        else:
            Total_variance = []
            for i in Portfolio:
                for j in Portfolio:
                    if i != j:
                        Total_variance = i[2]*j[2]*self.expected_covariance(i[0], j[0]) + Total_variance
                    if i == j:
                        Total_variance = i[2]**2*self.expected_variance(i[0]) + Total_variance
            return Total_variance
                                    
    #Gets expected return on a Portfolio
    def expected_return(self, Portfolio):
        Return = 0
        if len(Portfolio) == 0:
            return 0
        else:
            print(Portfolio)
            for i in Portfolio:
                print(getattr(i[0], "EPI_after_screening"))
                Projected_EPI = getattr(i[0], "EPI_after_screening")
                for j in range(0,(Early_startup_exit-getattr(i[0],"Life_stage"))):                    
                    Projected_EPI = self.time_progression_projected(Projected_EPI)
                    print(Projected_EPI)
                    if Projected_EPI < 0:
                        Projected_EPI = 0
                    if Projected_EPI > 1:
                        Projected_EPI = 0.99
                print("Projected_EPI is")
                print(Projected_EPI)
                print("EPI_to_returns")
                print(self.EPI_to_returns(Projected_EPI, i[1]))
                Return = float((self.EPI_to_returns(Projected_EPI, i[1])*i[2]))+ Return
                print("Return:")
                print(Return)
            return Return
    
    #Expected Sharpe ratio
    def expected_portfolio_coefficient(self, Portfolio):     
        return float(self.expected_return(Portfolio) - Risk_free_rate)/float(self.expected_portfolio_variance(Portfolio))
    
    #Is a startup in the portfolio or not
    def startup_in_portfolio(self, Prospect):
        for i in self.Portfolio:
            if Prospect[0] in i:
                return 1
            else:
                return 0
        
    #Gets reward after taking a particular action a 
    def get_reward(self, action, startup):
        #If less than 0.01 was invested, we assume that VC does not invest into a given startup
        if action <0.01:
            return troch.tensor([0])
        #If action in this range then VC invests in startup
        if 0.01<=action<=1:
            return torch.tensor([(self.expected_portfolio_coefficient((self.Portfolio + [list(startup) + list(action)])) - self.expected_portfolio_coefficient(self.Portfolio))])
        #
        if action>1:
            return torch.tensor([-100])
        
    #Gets state which is inputed into the RL model
    def get_state(self, Prospect):
        
        #Prospect attributes
        Startup_in_portfolio = 0
        if self.Portfolio_size !=0:
            Startup_in_portfolio = int(self.startup_in_portfolio(Prospect))
        Prospect_EPI = getattr(Prospect[0], "EPI_after_screening")
        Prospect_stage = 0 
        if Prospect[1] == "Late":
            Prospect_stage = 1
        Industry_correlation = 0
        for i in self.Portfolio:    
            Industry_correlation =  Correlation_matrix.loc[getattr(Prospect, "Industry"), getattr(i[0], "Industry")] + Industry_correlation
        Average_correlation_with_portfolio = 0
        if self.Portfolio_size != 0:
            Average_correlation_with_portfolio = Industry_correlation/self.Portfolio_size
        
        #Cohort attributes
        total_cohort = 0
        for i in self.Screening_prospects:
            total_cohort = getattr(i[0], "EPI_after_screening") + total_cohort
        Screenings_mean = total_cohort/len(self.Screening_prospects) 
        Screenings = []
        for i in self.Screening_prospects:
            Screenings.append(getattr(i[0], "EPI_after_screening"))
        Screenings_sd = np.std(Screenings)
            
        #Portfolio attributes
        total = 0
        for i in self.Portfolio:
            total = getattr(i[0], "EPI_after_screening") + total
        Portfolio_mean = 0
        if self.Portfolio_size != 0:
            Portfolio_mean = total/self.Portfolio_size
        EPIs = []
        Portfolio_sd = 0
        if self.Portfolio_size != 0:
            for i in self.Portfolio:
                EPIs.append(getattr(i[0], "EPI_after_screening"))
            Portfolio_sd = np.std(EPIs)
        
        state_ = torch.tensor([Startup_in_portfolio, Prospect_EPI, Prospect_stage, Average_correlation_with_portfolio, Screenings_mean, Screenings_sd, Portfolio_mean, Portfolio_sd, self.VC_quality, self.Endowement, self.Time_till_end_of_investment_stage_E, self.Time_till_end_of_investment_stage_L])
        return state_
    
    #Gets next state 
    def get_next_state(self, action, Prospect):
        Startup_in_portfolio = 0
        Prospect_EPI = 0
        Prospect_stage = 0 
        Average_correlation_with_portfolio = 0
        
        #Cohort attributes
        total_cohort = 0
        for i in self.Screening_prospects:
            total_cohort = getattr(i[0], "EPI_after_screening") + total_cohort
        Screenings_mean = total_cohort/len(self.Screening_prospects) 
        Screenings = []
        for i in self.Screening_prospects:
            Screenings.append(getattr(i[0], "EPI_after_screening"))
        Screenings_sd = np.std(Screenings)
            
        #Portfolio attributes
        total = 0
        for i in self.Portfolio:
            total = getattr(i[0], "EPI_after_screening") + total
        Portfolio_mean = 0
        if self.Portfolio_size != 0:
            Portfolio_mean = total/self.Portfolio_size
        EPIs = []
        Portfolio_sd = 0
        if self.Portfolio_size != 0:
            for i in self.Portfolio:
                EPIs.append(getattr(i[0], "EPI_after_screening"))
            Portfolio_sd = np.std(EPIs)
        
        next_state_ = torch.tensor([Startup_in_portfolio, Prospect_EPI, Prospect_stage, Average_correlation_with_portfolio, Screenings_mean, Screenings_sd, Portfolio_mean, Portfolio_sd, self.VC_quality, self.Endowement, self.Time_till_end_of_investment_stage_E, self.Time_till_end_of_investment_stage_L])
        return next_state_
    
    def step(self):
        for i in self.Screening_prospects:
            done = 0
            obs = self.get_state(i)
            act = agent.choose_action(obs)
            print(obs)
            print(act)
            print(self.Portfolio)
            new_state, reward = self.get_next_state(act, obs), self.get_reward(act, i)
            agent.remember(obs, act, reward, new_state, int(done))
            agent.learn()
            if 0.01<=act<=1:
                i[0].VC_potential_investments.append(self)
                self.Portfolio.append(i+[act])
        self.Fund_life_stage += 1
        
class Startup(Agent):
    def __init__(self, unique_id, EPI, Industry, Life_stage, model):
        self.unique_id = unique_id
        self.model = model
        self.Industry = Industry
        self.EPI = EPI
        self.Life_stage = Life_stage
        self.EPI_with_noise = 0
        self.EPI_after_screening = 0
        self.VC_potential_investments = []
        self.VC_investments = []
    
    def average_investor_quality(self):
        total = 0
        if len(self.VC_investments) != 0:
            for i in self.VC_investments:
                total = getattr(i[0], "VC_quality") + total
            return total/len(self.VC_investments)
        if len(self.VC_investments) == 0 :
            return 0
    
    #This is a funciton which makes a startup to progress over time                                
    #The EPI(potential) in time t+1 depends on EPI in time t, advising from VC and random shocks
    def time_progression(self):
        self.EPI = Alpha*self.EPI + Beta*self.average_investor_quality() + np.random.normal(Idiosyncratic_shock_mean, Idiosyncratic_shock_sd)\
        + np.random.normal(Industry_shock_mean, Industry_shock_sd) + np.random.normal(Macro_shock_mean, Macro_shock_sd)
        self.Life_stage += 1
                                    
    def noise_before_screening(self):
        if self.Life_stage == 0:
            self.EPI_with_noise = self.EPI + np.random.normal(Noise_mean_before_screening_ES, Noise_standard_deviation_before_screening_ES)
            while self.EPI_with_noise>1 or self.EPI_with_noise<0:
                self.EPI_with_noise = self.EPI + np.random.normal(Noise_mean_before_screening_ES, Noise_standard_deviation_before_screening_ES)
        else:
            self.EPI_with_noise = self.EPI + np.random.normal(Noise_mean_before_screening_ES, Noise_standard_deviation_before_screening_ES/(self.Life_stage**(1/2)))
            while self.EPI_with_noise>1 or self.EPI_with_noise<0:
                self.EPI_with_noise = self.EPI + np.random.normal(Noise_mean_before_screening_ES, Noise_standard_deviation_before_screening_ES/(self.Life_stage**(1/2)))
    
    def noise_after_screening(self):
        if self.Life_stage == 0:
            self.EPI_after_screening = self.EPI + np.random.normal(Noise_mean_after_screening_ES, Noise_standard_deviation_after_screening_ES)
            while self.EPI_after_screening>1 or self.EPI_after_screening<0:
                self.EPI_after_screening = self.EPI + np.random.normal(Noise_mean_after_screening_ES, Noise_standard_deviation_after_screening_ES)
        else:
            self.EPI_after_screening = self.EPI + np.random.normal(Noise_mean_after_screening_ES, Noise_standard_deviation_after_screening_ES/(self.Life_stage**(1/2)))
            while self.EPI_after_screening>1 or self.EPI_after_screening<0:
                self.EPI_after_screening = self.EPI + np.random.normal(Noise_mean_after_screening_ES, Noise_standard_deviation_after_screening_ES)
    
    #Function step refers to range of updates that occur every time step 
    def step(self):
        self.VC_potential_investments.sort(key = lambda x: x.VC_quality, reverse=True)
        for i in self.VC_potential_investments[:5]:
            self.VC_investments.append(i)
        self.VC_potential_investments = []
        #Updating EPI with noise for startups
        self.noise_before_screening()
        self.noise_after_screening()
        #Collecting the prospects for this time step, 
        #0.450 and 0.570 correspond to levels of EPI that give return greater than 2
        if self.Life_stage == 0 and self.EPI_with_noise > 0.450:
            world.Early_stage_prospects.append(self)
        if self.Life_stage == 8 and self.EPI_with_noise > 0.570:
            world.Late_stage_prospects.append(self) 
        self.time_progression()  
        #We also make all the startups progress in time    
       
        
#Activation class, determines in which order agents are activated
class Activation_1(BaseScheduler):
    def step(self):
        #First, starups are activated
        for agent in self.agent_buffer(shuffled=True):
            if agent.unique_id >= world.VC_number:
                agent.step()
                
class Activation_2(BaseScheduler):
    def step(self):
        #Then, VCs are activated
        for agent in self.agent_buffer(shuffled=True):
            if agent.unique_id < world.VC_number:
                agent.step()
        #After agents are activated, we updated the step value


In [328]:
class World(Model):
    def __init__(self, VC_number, Startup_number):
        self.VC_number = VC_number
        self.Startup_number = Startup_number
        self.schedule_1 = Activation_1(self) 
        self.schedule_2 = Activation_2(self)
        self.Early_stage_prospects = []
        self.Late_stage_prospects = []
        self.VCs = []
        
        #Creating Agents - VC
        for i in range (Number_of_VCs):
            a = VC(i,float(theoretical_distribution_VC.generate_random(1)/VC_max_TVPI),int(lognorm.rvs(Number_of_employees_sd, Number_of_employees_loc, Number_of_employees_scale)),0,self)
            while a.VC_quality>1:
                a.VC_quality = float(theoretical_distribution_VC.generate_random(1)/VC_max_TVPI)
            self.schedule_2.add(a)
            self.VCs.append(a)
            
        self.VCs.sort(key = attrgetter('VC_quality'), reverse = True)                
        
        ##Collecting data
        self.datacollector = DataCollector(
          agent_reporters={"EPI": lambda a: getattr(a, "EPI", None)}
        )

    #Creating Agents - Startups_early stage
    def create_startups_early_1(self):        
        for j in range (Number_of_VCs + self.schedule_1.steps*Number_of_new_startups + self.schedule_1.steps*Number_of_late_stage_startups, Number_of_VCs +  (self.schedule_1.steps+1)*Number_of_new_startups + self.schedule_1.steps*Number_of_late_stage_startups):
            b = Startup(j, powerlaw.rvs(EPI_alpha, EPI_loc, EPI_scale),random.choices(List_of_Industries, Probability_Distribution_of_Industries),0, self)
            self.schedule_1.add(b)         
                
    def create_startups_early_2(self):        
        for j in range (Number_of_VCs + self.schedule_1.steps*Number_of_new_startups + 8*Number_of_late_stage_startups, Number_of_VCs +  (self.schedule_1.steps+1)*Number_of_new_startups + 8*Number_of_late_stage_startups):
            b = Startup(j, powerlaw.rvs(EPI_alpha, EPI_loc, EPI_scale),random.choices(List_of_Industries, Probability_Distribution_of_Industries),0, self)       
            self.schedule_1.add(b) 
       
    #Creating Agents - Startups_late stage
    def create_startups_late(self):
        #Creating list of late_stage EPIs 
        Data = []
        for i in range(Number_of_new_startups):
            epi = powerlaw.rvs(EPI_alpha, EPI_loc, EPI_scale)
            epi_with_noise = epi + np.random.normal(Noise_mean_before_screening_ES, Noise_standard_deviation_before_screening_ES)
            while epi_with_noise >1 or epi_with_noise<0:
                epi_with_noise = epi + np.random.normal(Noise_mean_before_screening_ES, Noise_standard_deviation_before_screening_ES)
            epi_after_screening = epi + np.random.normal(Noise_mean_after_screening_ES, Noise_standard_deviation_after_screening_ES)
            while epi_after_screening >1 or epi_after_screening<0:
                epi_after_screening = epi + np.random.normal(Noise_mean_after_screening_ES, Noise_standard_deviation_after_screening_ES)
            data_point = [epi, epi_with_noise, epi_after_screening]
            if data_point[1] > 0.450:
                Data.append(data_point)
        Data.sort(key = lambda x: x[1], reverse = True)
        Data_selection = Data[:Estimate_of_early_stage_screenings]
        Data_selection.sort(key = lambda x: x[2], reverse = True)
        Data_selection_final = Data_selection[:Number_of_late_stage_startups]
        EPI_list = []
        for i in Data_selection_final:
            EPI_list.append(i[0])    
        for (j, i) in zip(range (Number_of_VCs + (self.schedule_1.steps+1)*Number_of_new_startups + self.schedule_1.steps*Number_of_late_stage_startups, Number_of_VCs + (self.schedule_1.steps+1)*Number_of_new_startups + (self.schedule_1.steps+1)*Number_of_late_stage_startups), range(Number_of_late_stage_startups)):
            c = Startup(j, EPI_list[i],random.choices(List_of_Industries, Probability_Distribution_of_Industries),8, self)
            self.schedule_1.add(c)
                
    def startups_generation(self):
        if self.schedule_1.steps<8:
            self.create_startups_early_1()
            self.create_startups_late()
        else:
            self.create_startups_early_2()
            
    def matching_prospects_to_VCs(self):
        index = 0
        for i in world.Early_stage_prospects:
            for j in world.VCs:
                if getattr(j, "Number_of_available_screenings")*(1-Percentage_of_startups_getting_to_later_stage) > 1+ len(getattr(j, "Screening_prospects")):
                    j.Screening_prospects.append([i, "Early"])
                    i.VC_investments.append([j])
                    index +=1
                if index == 10:
                    break
            index = 0
        index = 0 
        for i in world.Late_stage_prospects:
            for j in world.VCs:
                if getattr(j, "Number_of_available_screenings")*Percentage_of_startups_getting_to_later_stage > len(getattr(j, "Screening_prospects")):
                    j.Screening_prospects.append([i, "Late"])
                    i.VC_investments.append([j])
                    index +=1
                if index == 10:
                    break
            index = 0
        
    def step(self):
        self.Early_stage_prospects = []
        self.Late_stage_prospects = []
        self.startups_generation()
        self.schedule_1.step()
        self.Early_stage_prospects.sort(key = attrgetter('EPI_with_noise'), reverse = True)
        self.Late_stage_prospects.sort(key = attrgetter('EPI_with_noise'), reverse = True)
        self.matching_prospects_to_VCs()
        self.schedule_2.step()
        self.schedule_1.steps += 1
        self.schedule_1.time += 1
        #self.datacollector.collect(self)


        
        

In [329]:
world = World(Number_of_VCs, Number_of_new_startups)
for i in range(1):
    world.step()

tensor([ 0.0000,  0.2706,  0.0000,  0.0000,  0.3626,  0.2587,  0.0000,  0.0000,
         0.4162,  1.0000,  8.0000, 16.0000], dtype=torch.float64)
[0.50101143]
[]
[[<__main__.Startup object at 0x7fea76885880>, 'Early', 0.50101143]]
0.2706384409645302
0.24110232028655454
0.20116399771814114
0.236735706274221
0.268915265777841
0.04569361247791695
0.07980630955643991
0.035726539084599496
0.2786438071094748
0.1874350762371586
0.23349197483530265
0.3475284021531786
0.2986677661941524
0.08296533669732348
0.20587763824724628
0.17698593467534895
0.20001319787479993
0.24906968988941028
0.09210091819007853
0.06541125753435728
0.09623354558369246
0.11911428957835635
0.06490581513078365
0.0790331553316706
-0.05223945320240002
0.04641680191194631
0.07183260077227728
0.1052436126831563
0.31264543919969723
0.5318969754578955
0.4494594989614988
0.5205943572678097
Projected_EPI is
0.5205943572678097
EPI_to_returns
2.5766608221732867
Return:
1.2909365262783092


ZeroDivisionError: float division by zero

#### for i in world.VCs:
    print(i.Screening_prospects)

In [94]:
    def startup_in_portfolio(self, Prospect):
        for i in Portfolio:
            if Prospect[0] in i:
                return 1
            else:
                return 0 
    
    def get_state(self, Prospect):
        
        #Prospect attributes
        Startup_in_portfolio = self.startup_in_portfolio(Prospect)
        Prospect_EPI = getattr(Prospect[0], "EPI_after_screening")
        Prospect_stage = 0 
        if Prospect[1] == "Late":
            Prospect_stage = 1
        Industry_correlation = 0
        for i in self.Portfolio:    
            Industry_correlation =  Correlation_matrix.loc[getattr(Prospect, "Industry"), getattr(i[0], "Industry")] + Industry_correlation
        Average_correlation_with_portfolio = 0
        if self.Portfolio_size != 0:
            Average_correlation_with_portfolio = Industry_correlation/self.Portfolio_size
        
        #Cohort attributes
        total_cohort = 0
        for i in self.Screening_prospects:
            total_cohort = getattr(i[0], "EPI_after_screening") + total_cohort
        Screenings_mean = total_cohort/len(self.Screening_prospects) 
        Screenings = []
        for i in self.Screening_prospects:
            Screenings.append(getattr(i[0], "EPI_after_screening"))
        Screenings_sd = np.std(Screenings)
            
        #Portfolio attributes
        total = 0
        for i in self.Portfolio:
            total = getattr(i[0], "EPI_after_screening") + total
        Portfolio_mean = 0
        if self.Portfolio_size != 0:
            Portfolio_mean = total/self.Portfolio_size
        EPIs = []
        for i in self.Portfolio:
            EPIs.append(getattr(i[0], "EPI_after_screening"))
        Portfolio_sd = np.std(EPIs)
        
        state_ = torch.tensor([Startup_in_portfolio, Prospect_EPI, Prospect_stage, Average_correlation_with_portfolio, Screenings_mean, Screenings_sd, Portfolio_mean, Portfolio_sd, self.VC_quality, self.Endowement, self.Time_till_end_of_investment_stage_E, self.Time_till_end_of_investment_stage_L])
        return state_

In [6]:
class Car:
    def __init__(self, number):
        self.number = number

In [9]:
Cars = []
for i in range(10):
    s = Car(np.random.uniform(0,10))
    Cars.append(s)
    

In [15]:
for i in Cars:
    print(getattr(i,"number"))

3.3666887275630986
7.435953875404775
2.05151240438901
2.362214039784009
3.924088859831926
0.08103013361251077
4.961311873982178
3.091929139485692
5.2001717169607895
5.513971306273037


In [18]:
Cars.sort(key = lambda x: x.number, reverse=True)

In [20]:
for i in Cars:
    print(getattr(i,"number"))

7.435953875404775
5.513971306273037
5.2001717169607895
4.961311873982178
3.924088859831926
3.3666887275630986
3.091929139485692
2.362214039784009
2.05151240438901
0.08103013361251077


In [25]:
Cars_selection = Cars[:5]

In [26]:
Cars_selection

In [30]:
Cars_selection.append(i in Cars[:5])

In [31]:
for i in CaCars_selection

 False]

In [45]:
i = [0,3]

In [46]:
i+[1]

[0, 3, 1]

In [82]:
Startup_in_portfolio = 0
Prospect_EPI = 0.4
Prospect_stage = 1
Average_correlation_with_portfolio = 0.5
Screenings_mean = 0.2
Screenings_sd = 0.3
Portfolio_mean = 0
Portfolio_sd = 0
VC_quality = 0.8
Endowement = 1
Time_till_end_of_investment_stage_E = 4
Time_till_end_of_investment_stage_L = 6

state_ = torch.tensor([Startup_in_portfolio, Prospect_EPI, Prospect_stage, Average_correlation_with_portfolio, Screenings_mean, Screenings_sd, Portfolio_mean, Portfolio_sd, VC_quality, Endowement, Time_till_end_of_investment_stage_E, Time_till_end_of_investment_stage_L])


In [85]:
agent.choose_action(state_)

/Users/samuelgramata/Documents/00_Dissertation/Model/Model/Model/ddpg.py:208: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  observation = T.tensor(observation, dtype=T.float).to(self.actor.device)


array([0.519666], dtype=float32)

In [100]:
type(torch.tensor([2]))

torch.Tensor

In [126]:
    def startup_in_portfolio(self, Prospect):
        for i in self.Portfolio:
            if Prospect[0] in i:
                return 1
            else:
                return 0

In [ ]:
Portfolio = [()]

In [146]:
s = torch.tensor([ 0.0000,  0.2807,  0.0000,  0.0000,  0.3781,  0.2599,  0.0000,  0.0000,
         0.2353,  1.0000,  8.0000, 16.0000])

In [147]:
agent.choose_action(s)

array([nan], dtype=float32)

In [192]:
    def time_progression_projected(EPI):
        EPI_ = Alpha*EPI + Beta*0.8 + np.random.normal(Idiosyncratic_shock_mean, Idiosyncratic_shock_sd)\
        + np.random.normal(Industry_shock_mean, Industry_shock_sd) + np.random.normal(Macro_shock_mean, Macro_shock_sd)
        return EPI_

In [194]:
time_progression_projected(0.7)

0.7792028641755653

In [199]:
    def expected_return(self, Portfolio):
        Return = []
        for i in Portfolio:
            Projected_EPI = getattr(i[0], "EPI_with_noise")
            for j in range(0,(Early_startup_exit-getattr(i[0],"Life_stage"))):                    
                Projected_EPI = self.time_progression_projected(Projected_EPI)
            Return = (EPI_to_returns(Projected_EPI, i[1])*i[2])+ Return
        return Return

In [216]:
probability = 1.6
sample_data_ER[int(sample_size*probability)]

IndexError: list index out of range

In [284]:
a = [1,2,3]

In [285]:
s = [4]

In [286]:
print(a+s)

[1, 2, 3, 4]
